In [1]:
#Files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import glob
import pickle

#DATA
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg

#CNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16

#VIS
from keras.utils.vis_utils import plot_model

In [4]:
def _prepareData(path): 
    '''
    params: path(string)
    return: [list list] of images in dataset and the list of labels
    '''
    labelsList = []
    listOfimg = []
    for directory in sorted(glob.glob(os.path.join(path, '*')), key = lambda k: k.split("/")[-1]):
            for img in glob.glob(os.path.join(directory,'*.jpg')):
                imgcv = cv2.imread(img)
                imgcv_r = cv2.resize(imgcv,(224,224)) #Resize to 128,128
                listOfimg.append(imgcv_r)
                labelsList.append(int(directory.split("/")[-1].replace('c','')))
    
    X_Train, X_Test, Y_Train, Y_Test =  train_test_split(listOfimg,labelsList, test_size = 0.2)
    Y_Train = tf.keras.utils.to_categorical(Y_Train, num_classes=10)
    Y_Test = tf.keras.utils.to_categorical(Y_Test, num_classes=10)

    return np.array(X_Train), np.array(X_Test), Y_Train, Y_Test

In [5]:
'''#Paths
pathTrainImages = "/kaggle/input/state-farm-distracted-driver-detection/img/train/"
pathPropagateImages =  "/kaggle/input/state-farm-distracted-driver-detection/img/test/"

#List of Images for Train and Test
X_Train, X_Test, Y_Train, Y_Test = _prepareData(pathTrainImages)

print("Size X_Train: {}, Size Y_Train: {}".format(len(X_Train),len(Y_Train)))
print("Size X_Test: {}, Size Y_Test: {}".format(len(X_Test),len(Y_Test)))
'''
#Paths
pathTrain_Images = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train/"
pathPropagate_Images =  "/kaggle/input/state-farm-distracted-driver-detection/imgs/test/"

#List of Images for Train and Test
X_Train, X_Test, Y_Train, Y_Test = _prepareData(pathTrain_Images)

print("Size X_Train: {}, Size Y_Train: {}".format(len(X_Train),len(Y_Train)))
print("Size X_Test: {}, Size Y_Test: {}".format(len(X_Test),len(Y_Test)))

In [6]:
def vgg(input_shape, num_classes, mode):
    vgg = VGG16(
        include_top=False, weights='imagenet', input_tensor=None,
        input_shape=input_shape, pooling=None, classes=num_classes,
        classifier_activation='softmax'
        )
    if mode=='feature-extraction':
        vgg.trainable = False
    if mode=='fine-tune':
        vgg.trainable = True
        # Let's take a look to see how many layers are in the base model
        print("Number of layers in the base model: ", len(vgg.layers))

        # Fine-tune from this layer onwards
        fine_tune_at = 10

        # Freeze all the layers before the `fine_tune_at` layer
        for layer in vgg.layers[:fine_tune_at]:
            layer.trainable =  False
    print(vgg.summary())
    return vgg

In [7]:
def transfer_learn(base_model, input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
#     x = utils.augment_data(inputs)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [10]:
NUM_CLASSES = 10
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
INPUT_SHAPE = (224,224,3)
# weights_path = os.path.join(utils.BASE_PATH, 'weights/')
# history_path = os.path.join(utils.BASE_PATH, 'history/')
def compile_model(model, learning_rate):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    print(model.summary())
    return model


def fit_model(model, initial_epochs, train_data, train_labels):
    history = model.fit(x = train_data, y = train_labels,
                        epochs = initial_epochs, batch_size = 8,
                        verbose = 1,validation_split=0.2)
    return history

# def save_weights(model, name):
#     model.save(weights_path+name+".h5")

# def save_history(history, file_name):
#     with open(file_name, 'wb') as file_p:
#         pickle.dump(history.history, file_p)

In [ ]:
fine_tune_epochs = 10
total_epochs =  NUM_EPOCHS + fine_tune_epochs
vgg16_fine = vgg(INPUT_SHAPE, NUM_CLASSES, 'fine-tune')
fine_model = compile_model(transfer_learn(vgg16_fine, INPUT_SHAPE, NUM_CLASSES), LEARNING_RATE)
history = fit_model(fine_model, total_epochs, X_Train, Y_Train)
# save_weights(fine_model, 'fineTune')
# save_history(fine_history, history_path+'fineTune')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.ylim([0.9,1])
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.ylim([0,.4])
plt.legend(['train','test'], loc='upper left')
plt.show()